In [24]:
###########
TO DO
###########

# Should weight by number of shots (right now each player getting equal weight)
# In groupby get count as well as mean...

# Streaky score for each player (how predictive previous shots are of future shots)


SyntaxError: invalid syntax (<ipython-input-24-01dfa6b8182e>, line 2)

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/shot_logs.csv')

In [3]:
list(df.columns)

['GAME_ID',
 'MATCHUP',
 'LOCATION',
 'W',
 'FINAL_MARGIN',
 'SHOT_NUMBER',
 'PERIOD',
 'GAME_CLOCK',
 'SHOT_CLOCK',
 'DRIBBLES',
 'TOUCH_TIME',
 'SHOT_DIST',
 'PTS_TYPE',
 'SHOT_RESULT',
 'CLOSEST_DEFENDER',
 'CLOSEST_DEFENDER_PLAYER_ID',
 'CLOSE_DEF_DIST',
 'FGM',
 'PTS',
 'player_name',
 'player_id']

In [4]:
df.head(2)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148


In [5]:
# Select columns of interest
df = df[['player_name', 'GAME_ID', 'FGM']]

In [6]:
df.head(2)

,player_name,GAME_ID,FGM
0,brian roberts,21400899,1
1,brian roberts,21400899,0


## Shot History

In [7]:
df['shot_m1'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=1))
df['shot_m2'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=2))
df['shot_m3'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=3))
df['shot_m4'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=4))
df['shot_m5'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=5))
df['shot_m6'] = df.groupby(['player_name', 'GAME_ID'])['FGM'].apply(lambda x: x.shift(periods=6))

In [8]:
df.head()

,player_name,GAME_ID,FGM,shot_m1,shot_m2,shot_m3,shot_m4,shot_m5,shot_m6
0,brian roberts,21400899,1,NaN,NaN,NaN,NaN,NaN,NaN
1,brian roberts,21400899,0,1.0,NaN,NaN,NaN,NaN,NaN
2,brian roberts,21400899,0,0.0,1.0,NaN,NaN,NaN,NaN
3,brian roberts,21400899,0,0.0,0.0,1.0,NaN,NaN,NaN
4,brian roberts,21400899,0,0.0,0.0,0.0,1.0,NaN,NaN


## Min Shot Count

In [9]:
df.shape

(128069, 9)

In [10]:
grouped = df.groupby(['player_name', 'GAME_ID'])
df = grouped.filter(lambda x: x['FGM'].count()>9).reset_index(drop=True)

In [11]:
df.shape

(81588, 9)

In [12]:
# filter for min shots/game (10/15/20+)

# group by player and game
# min 10 shots
# was shot-1 good?
# player average shooting rate

# taking previous shot into account
    # column['shot_m1'] = duplicate the column and move up one
    # remove the first shot, then look at each shot
    
# player average for next shot if previous a hit    
    # hit_prev_miss = sum of hits after misses/sum of shots after misses
    # hit_prev_hit = sum of hits after hits/sum of shots after hits
    
# player average if previous shot a miss
    # miss_prev_miss = sum of misses after misses/sum of shots after misses
    # miss_prev_hit = sum of misses after hits/sum of shots after hits
    

# repeat for 2, 3, 4 shots

# check for 3 out of 4 previous shots


## Player p_stats

In [189]:
# get shooting percentage
p_stats = pd.DataFrame(df.groupby(['player_name'])['FGM'].mean()).reset_index()
p_stats = p_stats.rename(columns={'FGM' : 'FG_pct'})
p_stats.head()

# maybe add mean number of shots by player from original df?

,player_name,FG_pct
0,aaron brooks,0.420904
1,al farouq aminu,0.500000
2,al horford,0.519544
3,al jefferson,0.479592
4,alan anderson,0.487805


### Hit Previous 1 Shot

I am requiring that they ONLY hit 1 previous shot in a row.

In [190]:
pd.isna(df.shot_m2[2])

False

In [191]:
df.head()

,player_name,GAME_ID,FGM,shot_m1,shot_m2,shot_m3,shot_m4,shot_m5,shot_m6,prev5_count
0,brian roberts,21400768,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,brian roberts,21400768,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0
2,brian roberts,21400768,0,1.0,1.0,NaN,NaN,NaN,NaN,2.0
3,brian roberts,21400768,1,0.0,1.0,1.0,NaN,NaN,NaN,2.0
4,brian roberts,21400768,0,1.0,0.0,1.0,1.0,NaN,NaN,3.0


In [192]:
hot_1 = df.loc[(df.shot_m1==1) & ((df.shot_m2==0) | (pd.isna(df.shot_m2)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_1_make', 'count': '1_make_shots'})

In [193]:
hot_1

,player_name,FG_pct_1_make,1_make_shots
0,aaron brooks,0.481928,83
1,al farouq aminu,0.333333,6
2,al horford,0.500000,160
3,al jefferson,0.502513,199
4,alan anderson,0.363636,11
...,...,...,...
260,wesley matthews,0.426901,171
261,wilson chandler,0.375723,173
262,zach lavine,0.516129,31
263,zach randolph,0.500000,150


In [194]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='right', on='player_name')

In [195]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots
0,aaron brooks,0.420904,0.481928,83
1,al farouq aminu,0.500000,0.333333,6


### Missed Previous 1 Shot

In [196]:
# they missed their previous shot
# they hit the shot before that (or there was not a shot before that)
hot_1 = df.loc[df.shot_m1==0 & ((df.shot_m2==1) | (pd.isna(df.shot_m2)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_1_miss', 'count': '1_miss_shots'})

In [197]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [198]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186
1,al farouq aminu,0.500000,0.333333,6,0.555556,9


### Hit Previous 2 Shots

In [199]:
# hit two shots in a row
# shot before was a miss or NAN
hot_1 = df.loc[(df.shot_m1==1) & (df.shot_m2==1) & ((df.shot_m3==0) | (pd.isna(df.shot_m3)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_2_make', 'count': '2_make_shots'})

In [200]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [201]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0


### Missed Previous 2 Shots

In [202]:
hot_1 = df.loc[(df.shot_m1==0) & (df.shot_m2==0) & ((df.shot_m3==1) | (pd.isna(df.shot_m3)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_2_miss', 'count': '2_miss_shots'})

In [203]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [204]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0


### Hit Previous 3 Shots

In [205]:
hot_1 = df.loc[(df.shot_m1==1) & (df.shot_m2==1) & (df.shot_m3==1) & ((df.shot_m4==0) | (pd.isna(df.shot_m4)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_3_make', 'count': '3_make_shots'})

In [206]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [207]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,3_make_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,0.357143,14.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,0.500000,2.0


### Missed Previous 3 Shots

In [208]:
hot_1 = df.loc[(df.shot_m1==0) & (df.shot_m2==0) & (df.shot_m3==0) & ((df.shot_m4==1) | (pd.isna(df.shot_m4)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_3_miss', 'count': '3_miss_shots'})

In [209]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [210]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,3_make_shots,FG_pct_3_miss,3_miss_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,0.357143,14.0,0.375,24.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,0.500000,2.0,0.000,1.0


### Hit Previous 4 Shots

In [211]:
hot_1 = df.loc[(df.shot_m1==1) & (df.shot_m2==1) & (df.shot_m3==1) & (df.shot_m4==1) & ((df.shot_m5==0) | (pd.isna(df.shot_m5)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_4_make', 'count': '4_make_shots'})

In [212]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [213]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,3_make_shots,FG_pct_3_miss,3_miss_shots,FG_pct_4_make,4_make_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,0.357143,14.0,0.375,24.0,0.4,5.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,0.500000,2.0,0.000,1.0,0.0,1.0


### Missed Previous 4 Shots

In [214]:
hot_1 = df.loc[(df.shot_m1==0) & (df.shot_m2==0) & (df.shot_m3==0) & (df.shot_m4==0) & ((df.shot_m5==1) | (pd.isna(df.shot_m5)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_4_miss', 'count': '4_miss_shots'})

In [215]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [216]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,3_make_shots,FG_pct_3_miss,3_miss_shots,FG_pct_4_make,4_make_shots,FG_pct_4_miss,4_miss_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,0.357143,14.0,0.375,24.0,0.4,5.0,0.333333,15.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,0.500000,2.0,0.000,1.0,0.0,1.0,NaN,NaN


### Hit Previous 5 Shots

In [217]:
hot_1 = df.loc[(df.shot_m1==1) & (df.shot_m2==1) & (df.shot_m3==1) & (df.shot_m4==1) & (df.shot_m5==1) & ((df.shot_m6==0) | (pd.isna(df.shot_m6)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_5_make', 'count': '5_make_shots'})

In [218]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [219]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,3_make_shots,FG_pct_3_miss,3_miss_shots,FG_pct_4_make,4_make_shots,FG_pct_4_miss,4_miss_shots,FG_pct_5_make,5_make_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,0.357143,14.0,0.375,24.0,0.4,5.0,0.333333,15.0,0.0,1.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,0.500000,2.0,0.000,1.0,0.0,1.0,NaN,NaN,NaN,NaN


### Missed Previous 5 Shots

In [220]:
hot_1 = df.loc[(df.shot_m1==0) & (df.shot_m2==0) & (df.shot_m3==0) & (df.shot_m4==0)& (df.shot_m5==0) & ((df.shot_m6==1) | (pd.isna(df.shot_m6)))]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_5_miss', 'count': '5_miss_shots'})

In [221]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [222]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,...,FG_pct_3_miss,3_miss_shots,FG_pct_4_make,4_make_shots,FG_pct_4_miss,4_miss_shots,FG_pct_5_make,5_make_shots,FG_pct_5_miss,5_miss_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,...,0.375,24.0,0.4,5.0,0.333333,15.0,0.0,1.0,0.1,10.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,...,0.000,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Hit 4 of 5 previous

In [223]:
df['prev5_count'] = df.loc[:,'shot_m1':'shot_m5'].sum(axis=1)

In [224]:
hot_1 = df.loc[df.prev5_count==4]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_80_pct', 'count': '80_pct_shots'})

In [225]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [226]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,...,FG_pct_4_make,4_make_shots,FG_pct_4_miss,4_miss_shots,FG_pct_5_make,5_make_shots,FG_pct_5_miss,5_miss_shots,FG_pct_80_pct,80_pct_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,...,0.4,5.0,0.333333,15.0,0.0,1.0,0.1,10.0,0.304348,23.0
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.0


### Missed 4 of 5 previous

In [227]:
hot_1 = df.loc[df.prev5_count==1]
hot_1 = pd.DataFrame(hot_1.groupby(['player_name'])['FGM'].agg(['mean', 'count'])).reset_index()
hot_1 = hot_1.rename(columns={'mean':'FG_pct_20_pct', 'count': '20_pct_shots'})

In [228]:
# add to p_stats df
p_stats = p_stats.merge(hot_1, how='left', on='player_name')

In [229]:
p_stats.head(2)

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,...,FG_pct_4_miss,4_miss_shots,FG_pct_5_make,5_make_shots,FG_pct_5_miss,5_miss_shots,FG_pct_80_pct,80_pct_shots,FG_pct_20_pct,20_pct_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,...,0.333333,15.0,0.0,1.0,0.1,10.0,0.304348,23.0,0.430233,86
1,al farouq aminu,0.500000,0.333333,6,0.555556,9,1.000000,2.0,0.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.0,0.666667,3


# Summary Stats

In [230]:
p_stats.describe()

,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,FG_pct_3_make,...,FG_pct_4_miss,4_miss_shots,FG_pct_5_make,5_make_shots,FG_pct_5_miss,5_miss_shots,FG_pct_80_pct,80_pct_shots,FG_pct_20_pct,20_pct_shots
count,265.000000,265.000000,265.000000,265.000000,265.000000,261.000000,261.000000,263.000000,263.000000,249.000000,...,224.000000,224.000000,180.000000,180.000000,195.000000,195.000000,249.000000,249.000000,265.000000,265.000000
mean,0.470284,0.470157,76.535849,0.486868,153.630189,0.455019,33.413793,0.480122,39.034221,0.454634,...,0.460201,10.424107,0.424582,3.244444,0.447957,5.733333,0.423965,27.224900,0.504096,72.188679
std,0.069348,0.125869,70.820406,0.098687,144.792298,0.177345,30.982955,0.180514,36.598226,0.238268,...,0.269124,9.220447,0.346308,2.551420,0.325632,4.804494,0.194142,26.300942,0.125185,66.086851
min,0.300000,0.000000,1.000000,0.166667,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.142857,1.000000
25%,0.423611,0.408654,15.000000,0.428571,28.000000,0.377778,8.000000,0.400000,8.000000,0.344828,...,0.321481,2.000000,0.000000,1.000000,0.200000,1.500000,0.333333,6.000000,0.437500,14.000000
50%,0.462019,0.466667,49.000000,0.468391,100.000000,0.444444,21.000000,0.473684,24.000000,0.461538,...,0.473389,8.000000,0.400000,2.000000,0.454545,5.000000,0.431818,17.000000,0.482759,48.000000
75%,0.500560,0.520000,128.000000,0.530686,253.000000,0.520833,56.000000,0.549038,64.000000,0.533333,...,0.600000,16.250000,0.600000,5.000000,0.630682,8.500000,0.510638,43.000000,0.538462,123.000000
max,0.700000,1.000000,258.000000,1.000000,542.000000,1.000000,121.000000,1.000000,144.000000,1.000000,...,1.000000,36.000000,1.000000,14.000000,1.000000,21.000000,1.000000,129.000000,1.000000,241.000000


## Export CSV

In [231]:
# filter out players with NaNs...lower usage players
p_stats.shape

(265, 26)

In [232]:
p_stats_slx = p_stats.dropna().reset_index(drop=True)

In [233]:
p_stats_slx.shape

(154, 26)

In [177]:
p_stats_slx.to_csv('cold-hand/data/nba_stats_2015_16_count.csv')

In [234]:
# make long
p_stats_slx_long = pd.melt(p_stats_slx, id_vars =['player_name'], value_vars =list(p_stats_slx.columns[1:-4]))

In [179]:
p_stats_slx_long.to_csv('cold-hand/data/nba_stats_2015_16_long_count.csv')

In [251]:
a = p_stats_slx_long.loc[p_stats_slx_long.variable.isin(['1_make_shots', '1_miss_shots', '2_make_shots', '2_miss_shots',
       '3_make_shots', '3_miss_shots', '4_make_shots', '4_miss_shots',
       '5_make_shots', '5_miss_shots'])].reset_index(drop=True)
a = a.rename(columns={'variable':'shot_streak', 'value': 'shot_count'})
a

,player_name,shot_streak,shot_count
0,aaron brooks,1_make_shots,83.0
1,al horford,1_make_shots,160.0
2,al jefferson,1_make_shots,199.0
3,amare stoudemire,1_make_shots,40.0
4,amir johnson,1_make_shots,28.0
...,...,...,...
1535,wesley johnson,5_miss_shots,8.0
1536,wesley matthews,5_miss_shots,8.0
1537,wilson chandler,5_miss_shots,15.0
1538,zach lavine,5_miss_shots,1.0


In [253]:
b = p_stats_slx_long.loc[p_stats_slx_long.variable.isin(['FG_pct_1_make', 'FG_pct_1_miss',
       'FG_pct_2_make', 'FG_pct_2_miss', 'FG_pct_3_make', 'FG_pct_3_miss',
       'FG_pct_4_make', 'FG_pct_4_miss', 'FG_pct_5_make', 'FG_pct_5_miss',
       ])].reset_index(drop=True)
b = a.rename(columns={'variable':'shot_streak', 'value': 'fg_pct'})
b

,player_name,shot_streak,shot_count
0,aaron brooks,1_make_shots,83.0
1,al horford,1_make_shots,160.0
2,al jefferson,1_make_shots,199.0
3,amare stoudemire,1_make_shots,40.0
4,amir johnson,1_make_shots,28.0
...,...,...,...
1535,wesley johnson,5_miss_shots,8.0
1536,wesley matthews,5_miss_shots,8.0
1537,wilson chandler,5_miss_shots,15.0
1538,zach lavine,5_miss_shots,1.0


In [248]:
p_stats_slx.columns.sort_values()

Index(['1_make_shots', '1_miss_shots', '20_pct_shots', '2_make_shots',
       '2_miss_shots', '3_make_shots', '3_miss_shots', '4_make_shots',
       '4_miss_shots', '5_make_shots', '5_miss_shots', '80_pct_shots',
       'FG_pct', 'FG_pct_1_make', 'FG_pct_1_miss', 'FG_pct_20_pct',
       'FG_pct_2_make', 'FG_pct_2_miss', 'FG_pct_3_make', 'FG_pct_3_miss',
       'FG_pct_4_make', 'FG_pct_4_miss', 'FG_pct_5_make', 'FG_pct_5_miss',
       'FG_pct_80_pct', 'player_name'],
      dtype='object')

In [186]:
p_stats_slx

,player_name,FG_pct,FG_pct_1_make,1_make_shots,FG_pct_1_miss,1_miss_shots,FG_pct_2_make,2_make_shots,FG_pct_2_miss,2_miss_shots,...,FG_pct_4_miss_y,4_miss_shots_y,FG_pct_5_make,5_make_shots,FG_pct_5_miss,5_miss_shots,FG_pct_80_pct,80_pct_shots,FG_pct_20_pct,20_pct_shots
0,aaron brooks,0.420904,0.481928,83,0.392473,186,0.432432,37.0,0.263158,38.0,...,0.333333,15.0,0.000000,1.0,0.100000,10.0,0.304348,23.0,0.430233,86
1,al horford,0.519544,0.500000,160,0.530686,277,0.535211,71.0,0.514706,68.0,...,0.300000,10.0,0.600000,5.0,0.500000,6.0,0.571429,77.0,0.520000,125
2,al jefferson,0.479592,0.502513,199,0.500000,382,0.423913,92.0,0.489796,98.0,...,0.550000,20.0,0.400000,5.0,0.750000,8.0,0.432099,81.0,0.458101,179
3,amare stoudemire,0.558442,0.550000,40,0.615385,65,0.428571,21.0,0.428571,14.0,...,0.000000,1.0,0.333333,3.0,1.000000,1.0,0.625000,16.0,0.538462,26
4,amir johnson,0.600000,0.607143,28,0.658537,41,0.625000,16.0,0.777778,9.0,...,0.000000,1.0,1.000000,1.0,1.000000,1.0,0.571429,21.0,0.500000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,wesley johnson,0.415584,0.431373,51,0.403226,124,0.600000,20.0,0.406250,32.0,...,0.200000,10.0,0.000000,1.0,0.625000,8.0,0.380952,21.0,0.464286,56
150,wesley matthews,0.452269,0.426901,171,0.504644,323,0.304348,69.0,0.543210,81.0,...,0.437500,16.0,0.333333,3.0,0.250000,8.0,0.416667,36.0,0.539062,128
151,wilson chandler,0.416792,0.375723,173,0.449438,356,0.379310,58.0,0.481481,81.0,...,0.291667,24.0,0.500000,2.0,0.333333,15.0,0.295455,44.0,0.415205,171
152,zach lavine,0.423611,0.516129,31,0.400000,75,0.375000,16.0,0.476190,21.0,...,0.600000,5.0,1.000000,1.0,0.000000,1.0,0.444444,9.0,0.485714,35


In [91]:
# prep data for connected dot plot in Flourish


,player_name,FG_pct,FG_pct_1_make,FG_pct_1_miss,FG_pct_2_make,FG_pct_2_miss,FG_pct_3_make,FG_pct_3_miss,FG_pct_4_make,FG_pct_4_miss,FG_pct_5_make,FG_pct_5_miss,FG_pct_80_pct,FG_pct_20_pct
0,aaron brooks,0.420904,0.481928,0.392473,0.432432,0.310680,0.350000,0.338462,0.333333,0.317073,0.000000,0.307692,0.304348,0.430233
1,al horford,0.519544,0.500000,0.530686,0.535211,0.533333,0.516129,0.557692,0.428571,0.428571,0.600000,0.545455,0.571429,0.520000
2,al jefferson,0.479592,0.502513,0.500000,0.423913,0.514286,0.396825,0.545455,0.320000,0.562500,0.375000,0.583333,0.432099,0.458101
3,amare stoudemire,0.558442,0.550000,0.615385,0.428571,0.565217,0.625000,0.777778,0.625000,0.500000,0.250000,1.000000,0.625000,0.538462
4,amir johnson,0.600000,0.607143,0.658537,0.625000,0.692308,0.437500,0.500000,0.333333,0.500000,0.500000,1.000000,0.571429,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,wesley johnson,0.415584,0.431373,0.403226,0.600000,0.411765,0.312500,0.416667,0.200000,0.450000,0.000000,0.700000,0.380952,0.464286
150,wesley matthews,0.452269,0.426901,0.504644,0.304348,0.506849,0.483871,0.461538,0.333333,0.406250,0.250000,0.375000,0.416667,0.539062
151,wilson chandler,0.416792,0.375723,0.449438,0.379310,0.413408,0.343750,0.357143,0.272727,0.362069,0.333333,0.411765,0.295455,0.415205
152,zach lavine,0.423611,0.516129,0.400000,0.375000,0.473684,0.545455,0.470588,0.666667,0.500000,0.750000,0.333333,0.444444,0.485714


In [65]:
from scipy import stats

In [237]:
p_stats.FG_pct_1_miss

0      0.392473
1      0.555556
2      0.530686
3      0.500000
4      0.555556
         ...   
260    0.504644
261    0.449438
262    0.400000
263    0.483221
264    0.447761
Name: FG_pct_1_miss, Length: 265, dtype: float64

---
**NOTE**: For now this is unweighted by the number of shots...treating all players as equal...

In [68]:
p_stats

,player_name,FG_pct,FG_pct_1_make,FG_pct_1_miss,FG_pct_2_make_x,FG_pct_2_make_y,FG_pct_2_miss,FG_pct_3_make,FG_pct_3_miss,FG_pct_4_make,FG_pct_4_miss,FG_pct_5_make,FG_pct_5_miss,FG_pct_80_pct,FG_pct_20_pct
0,aaron brooks,0.420904,0.481928,0.392473,0.432432,0.432432,0.263158,0.357143,0.375000,0.400000,0.333333,0.000000,0.100000,0.304348,0.430233
1,al farouq aminu,0.500000,0.333333,0.555556,1.000000,1.000000,0.500000,0.500000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.666667
2,al horford,0.519544,0.500000,0.530686,0.535211,0.535211,0.514706,0.588235,0.645161,0.333333,0.300000,0.600000,0.500000,0.571429,0.520000
3,al jefferson,0.479592,0.502513,0.500000,0.423913,0.423913,0.489796,0.447368,0.533333,0.294118,0.550000,0.400000,0.750000,0.432099,0.458101
4,alan anderson,0.487805,0.363636,0.555556,0.500000,0.500000,0.800000,0.500000,0.000000,1.000000,1.000000,0.000000,NaN,0.333333,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,wesley matthews,0.452269,0.426901,0.504644,0.304348,0.304348,0.543210,0.578947,0.515152,0.375000,0.437500,0.333333,0.250000,0.416667,0.539062
261,wilson chandler,0.416792,0.375723,0.449438,0.379310,0.379310,0.481481,0.380952,0.350000,0.250000,0.291667,0.500000,0.333333,0.295455,0.415205
262,zach lavine,0.423611,0.516129,0.400000,0.375000,0.375000,0.476190,0.400000,0.444444,0.500000,0.600000,1.000000,0.000000,0.444444,0.485714
263,zach randolph,0.492114,0.500000,0.483221,0.561644,0.561644,0.434783,0.461538,0.333333,0.666667,0.500000,0.500000,0.666667,0.539474,0.489510


In [63]:
len(p_stats.FG_pct_1_miss)

265

In [117]:
stats.ttest_ind(p_stats.FG_pct_1_miss, p_stats.FG_pct_1_make)

Ttest_indResult(statistic=1.7008149743987107, pvalue=0.08956654466377208)

In [118]:
stats.ttest_ind(p_stats.FG_pct_2_miss, p_stats.FG_pct_2_make, nan_policy='omit')

Ttest_indResult(statistic=1.605635656400505, pvalue=0.10895873897568766)

In [119]:
stats.ttest_ind(p_stats.FG_pct_3_miss, p_stats.FG_pct_3_make, nan_policy='omit')

Ttest_indResult(statistic=2.697382946838833, pvalue=0.007225192140797035)

In [120]:
stats.ttest_ind(p_stats.FG_pct_4_miss, p_stats.FG_pct_4_make, nan_policy='omit')

Ttest_indResult(statistic=1.6182382676031941, pvalue=0.10632298373349536)

In [121]:
stats.ttest_ind(p_stats.FG_pct_5_miss, p_stats.FG_pct_5_make, nan_policy='omit')

Ttest_indResult(statistic=0.6736335900832562, pvalue=0.5009614899480914)

In [122]:
stats.ttest_ind(p_stats.FG_pct_80_pct, p_stats.FG_pct_20_pct, nan_policy='omit')

Ttest_indResult(statistic=-5.594408209699447, pvalue=3.6096620244384676e-08)

In [246]:
p_stats

,player_name,FG_pct,FG_pct_1_make,FG_pct_1_miss,FG_pct_2_make,FG_pct_2_miss,FG_pct_3_make,FG_pct_3_miss,FG_pct_4_make,FG_pct_4_miss,FG_pct_5_make,FG_pct_5_miss,FG_pct_80_pct,FG_pct_20_pct
0,aaron brooks,0.420904,0.450000,0.392473,0.403509,0.310680,0.350000,0.338462,0.333333,0.317073,0.000000,0.307692,0.304348,0.430233
1,al farouq aminu,0.500000,0.454545,0.555556,0.600000,0.333333,0.333333,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.666667
2,al horford,0.519544,0.511945,0.530686,0.526316,0.533333,0.516129,0.557692,0.428571,0.428571,0.600000,0.545455,0.571429,0.520000
3,al jefferson,0.479592,0.463277,0.500000,0.412903,0.514286,0.396825,0.545455,0.320000,0.562500,0.375000,0.583333,0.432099,0.458101
4,alan anderson,0.487805,0.421053,0.555556,0.500000,0.714286,0.500000,0.500000,0.500000,1.000000,0.000000,NaN,0.333333,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,wesley matthews,0.452269,0.402214,0.504644,0.360000,0.506849,0.483871,0.461538,0.333333,0.406250,0.250000,0.375000,0.416667,0.539062
261,wilson chandler,0.416792,0.372624,0.449438,0.366667,0.413408,0.343750,0.357143,0.272727,0.362069,0.333333,0.411765,0.295455,0.415205
262,zach lavine,0.423611,0.482759,0.400000,0.444444,0.473684,0.545455,0.470588,0.666667,0.500000,0.750000,0.333333,0.444444,0.485714
263,zach randolph,0.492114,0.515464,0.483221,0.531915,0.431655,0.500000,0.428571,0.551724,0.529412,0.428571,0.583333,0.539474,0.489510


In [247]:
league_averages = pd.read_csv('cold-hand/data/NBA_league_averages.csv')

In [249]:
league_averages.reset_index()

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,Per Game,Per Game.1,Per Game.2,...,Per Game.15,Shooting,Shooting.1,Shooting.2,Advanced,Advanced.1,Advanced.2,Advanced.3,Advanced.4,Advanced.5
0,Rk,Season,Lg,Age,Ht,Wt,G,MP,FG,FGA,...,PTS,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
1,1,2020-21,NBA,26.2,6-6,217,842,241.4,41.1,88.3,...,111.8,.466,.367,.777,99.2,.537,12.5,22.2,.192,112.1
2,2,2019-20,NBA,26.1,6-6,218,1059,241.8,40.9,88.8,...,111.8,.460,.358,.773,100.3,.529,12.8,22.5,.201,110.6
3,3,2018-19,NBA,26.3,6-6,219,1230,241.6,41.1,89.2,...,111.2,.461,.355,.766,100.0,.524,12.4,22.9,.198,110.4
4,4,2017-18,NBA,26.4,6-7,220,1230,241.4,39.6,86.1,...,106.3,.460,.362,.767,97.3,.521,13.0,22.3,.193,108.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71,1950-51,NBA,NaN,NaN,NaN,354,NaN,29.8,83.6,...,84.1,.357,NaN,.733,NaN,.357,NaN,NaN,.293,NaN
72,72,1949-50,NBA,NaN,NaN,NaN,561,NaN,28.2,83.1,...,80.0,.340,NaN,.714,NaN,.340,NaN,NaN,.284,NaN
73,73,1948-49,BAA,NaN,NaN,NaN,360,NaN,29.0,88.7,...,80.0,.327,NaN,.703,NaN,.327,NaN,NaN,.248,NaN
74,74,1947-48,BAA,NaN,NaN,NaN,192,NaN,27.2,96.0,...,72.7,.284,NaN,.675,NaN,.284,NaN,NaN,.190,NaN
